In [1]:
import numpy as np

## Common Emitter Analysis

### Basic Voltage Divider CE amplifier

In [2]:
class CE_AMP_calculator:
    def __init__(self, Vcc, Ic, Vbe, B, fmin, Rs):
        self.Vcc = Vcc
        self.Ic = Ic
        self.Vbe = Vbe
        self.B = B
        self.fmin = fmin
        self.Rs = Rs

        self.Ve = 0.1 * Vcc
        # Load point is selected on center point of load line.
        self.Vce = Vcc / 2


    def _calculate_circuit_resistors(self):
        # 1. Resistors (based on design targets)
        self.Re = self.Ve / self.Ic
        self.Rc = (self.Vcc - self.Vce - self.Ve) / self.Ic

        # 2. Bias Resistors (10:1 divider rule)
        Vb = self.Ve + self.Vbe
        Ib = self.Ic / self.B
        Idiv = 10 * Ib

        self.R2 = Vb / Idiv
        self.R1 = (self.Vcc - Vb) / Idiv

        return {'Re':self.Re, 'Rc': self.Rc, 'R1': self.R1, 'R2': self.R2}

    def _calculate_input_voltage_limits(self, Rl, Av, Vce_sat):
        V_peak_sat = self.Vce - Vce_sat
        Rac = (1/self.Rc + 1/Rl)**-1
        V_peak_cutoff = self.Ic * Rac
        Vp_max = np.min([V_peak_sat, V_peak_cutoff])
        Vin_max = Vp_max / np.abs(Av)
        return Vin_max

    def _calculate_AC_parameters(self, Rl, Ie_actual=None, Vce_sat=0.2): #simulatd Ie
        if not hasattr(self, 'R1'):
            self._calculate_circuit_resistors()

        Ie = Ie_actual if Ie_actual is not None else self.Ic

        # --- AC Small-Signal Parameters ---
        # Dynamic emitter resistance
        re_ac = 25e-3 / Ie


        # Thevenin Resistance of bias network
        Rth = (1/self.R1 + 1/self.R2)**-1

        # --- C_E Calculation ---
        # Resistance looking into the base reflected to the emitter:
        # R_reflected = (Rth || Rs) / B
        if self.Rs == 0:
            R_parallel_base = Rth # Rth || infinity is Rth (avoid division by zero if R_S = 0)
        else:
            R_parallel_base = (1/Rth + 1/self.Rs)**-1

        # R'_E = R_E || (re_ac + R_reflected)
        R_reflected_to_emitter = re_ac + R_parallel_base / self.B
        Re_ac = (1 / self.Re + 1 / R_reflected_to_emitter)**-1
        # Capacitor formula for f_L = 1 / (2*pi*R*C)
        Ce = 1 / (2 * np.pi * self.fmin * Re_ac)

        # --- C_in Calculation ---
        # Amplifier Input Impedance: Zin = Rth || Zin_base
        Zin_base = self.B * re_ac
        Zin = (1/Zin_base + 1/Rth)**-1

        Rin_total = Zin + self.Rs
        Cin = 1 / (2*np.pi*self.fmin*Rin_total)

        # --- C_out Calculation ---
        # Amplifier Output Impedance: Zout = Rc
        Zout = self.Rc
        Rout_total = Zout + Rl
        Cout = 1 / (2*np.pi*self.fmin*Rout_total)

        # Theoretical voltage gain
        Rc_total = (1/self.Rc + 1/Rl)**-1
        Av = -Rc_total / re_ac

        # Calculating input sine permisible magnitude
        Vin_max = self._calculate_input_voltage_limits(Rl, Av, Vce_sat)

        return {'Ce': Ce, 'Cin': Cin, 'Cout': Cout, 'Av': Av, 'Vin_max': Vin_max}

    def perform_frequency_analysis(self, Rl, Cin, Cout, Ce):
        # 1. AC Emitter Resistance
        re_ac = 25e-3 / self.Ic

        # 2. Midband Voltage Gain (Am)
        Rc_total = (1/self.Rc + 1/Rl)**-1  # Rc || Rl
        Am = -Rc_total / re_ac             # Correct gain formula
        Am_dB = 20 * np.log10(np.abs(Am))  # Gain in dB (absolute value is used)

        # --- 3. Corner Frequency due to Cin (fL1) ---
        Rin_transistor = self.B * re_ac

        # R1 || R2 || Rin_transistor
        R_eq_base = (1/self.R1 + 1/self.R2 + 1/Rin_transistor)**-1

        # Resistance seen by Cin: Rs + (R1 || R2 || Rin_transistor)
        Rin_base = self.Rs + R_eq_base
        fL1 = 1 / (2 * np.pi * Cin * Rin_base)

        # --- 4. Corner Frequency due to Cout (fL2) ---
        # Resistance seen by Cout: Rc + Rl (Standard approximation, neglecting r_o)
        Rout_collector = self.Rc + Rl
        fL2 = 1 / (2 * np.pi * Cout * Rout_collector)

        # --- 5. Corner Frequency due to Ce (fLE) ---

        # R_th_base = Rs || R1 || R2
        R_th_base = (1/self.R1 + 1/self.R2 + 1/self.Rs)**-1

        # R_reflected = re' + (R_th_base / beta)
        R_reflected = re_ac + R_th_base / self.B

        # Req_emitter = Re || R_reflected
        Req_emitter = (1/self.Re + 1/R_reflected)**-1
        fLE = 1 / (2 * np.pi * Ce * Req_emitter)

        # 6. Total Low Cutoff Frequency (fL)
        fL_total = (fL1**2 + fL2**2 + fLE**2)**0.5

        return {
            'Midband Voltage Gain (Am)': Am,
            'Midband Voltage Gain (dB)': Am_dB,
            'fL1 (Cin)': fL1,
            'fL2 (Cout)': fL2,
            'fLE (Ce)': fLE,
            'Low Cutoff Frequency (fL)': fL_total
            }


In [3]:
calc = CE_AMP_calculator(
    Vcc=12,
    Ic=1e-3,
    Vbe=0.7,
    B=300,
    fmin=20,
    Rs=50
)

In [4]:
resistors = calc._calculate_circuit_resistors()
for k, v in resistors.items():
    print(f'{k}: {round(v/1e3, 2)}kΩ')

Re: 1.2kΩ
Rc: 4.8kΩ
R1: 303.0kΩ
R2: 57.0kΩ


In [5]:
ac_params = calc._calculate_AC_parameters(
    Rl = 1e4
)
ac_params

{'Ce': 0.00032283550983851336,
 'Cin': 1.2175200339261203e-06,
 'Cout': 5.376856185537004e-07,
 'Av': -129.7297297297297,
 'Vin_max': np.float64(0.025)}

In [6]:
calc.perform_frequency_analysis(1e4, ac_params['Cin'], ac_params['Cout'], ac_params['Ce'])

{'Midband Voltage Gain (Am)': -129.7297297297297,
 'Midband Voltage Gain (dB)': np.float64(42.260790266171846),
 'fL1 (Cin)': 20.0,
 'fL2 (Cout)': 20.0,
 'fLE (Ce)': 20.0,
 'Low Cutoff Frequency (fL)': 34.64101615137755}

### Negative feedback Emitter resistance Split

In [7]:
import numpy as np

class CE_AMP_calculator_NF:
    def __init__(self, Vcc, Ic, Vbe, B, fmin, Rs):
        self.Vcc = Vcc
        self.Ic = Ic
        self.Vbe = Vbe
        self.B = B
        self.fmin = fmin
        self.Rs = Rs

        self.Ve = 0.1 * Vcc
        # Load point is selected on center point of load line.
        self.Vce = Vcc / 2


    def _calculate_circuit_resistors(self):
        # 1. DC Resistors
        self.Re = self.Ve / self.Ic
        self.Rc = (self.Vcc - self.Vce - self.Ve) / self.Ic

        # 2. Bias Resistors (10:1 divider rule)
        Vb = self.Ve + self.Vbe
        Ib = self.Ic / self.B
        Idiv = 10 * Ib

        self.R2 = Vb / Idiv
        self.R1 = (self.Vcc - Vb) / Idiv

        return {'Re':self.Re, 'Rc': self.Rc, 'R1': self.R1, 'R2': self.R2}

    def _calculate_input_voltage_limits(self, Rl, Av, Vce_sat=0.2):
        V_peak_sat = self.Vce - Vce_sat
        Rac = (1/self.Rc + 1/Rl)**-1
        V_peak_cutoff = self.Ic * Rac
        Vp_max = np.min([V_peak_sat, V_peak_cutoff])
        Vin_max = Vp_max / np.abs(Av)
        return Vin_max

    def _calculate_AC_parameters(self, Rl, Av_new, Ie_actual=None, Vce_sat=0.2):
        # Design function: Calculates R_E1, R_E2, and required Capacitors for target Av_new
        if not hasattr(self, 'R1'):
            self._calculate_circuit_resistors()

        Ie = Ie_actual if Ie_actual is not None else self.Ic
        re_ac = 25e-3 / Ie
        Rth = (1/self.R1 + 1/self.R2)**-1

        # Calculate R_C || R_L
        Rc_total = (1/self.Rc + 1/Rl)**-1

        # Design R_E1 based on target gain: Av = Rc_total / (re_ac + Re1)
        Re1 = (Rc_total / np.abs(Av_new)) - re_ac
        Re2 = self.Re - Re1

        # Check for valid resistor split
        if Re1 < 0 or Re2 < 0:
            raise ValueError("Invalid Av_new. The calculated Re1 or Re2 is negative. Target gain is too high or DC bias Re is too small.")

        # --- C_in Calculation ---
        # Zin_base = Beta * (re' + Re1)
        Zin_base = self.B * (re_ac + Re1)
        Zin = (1/Zin_base + 1/Rth)**-1

        Rin_total = Zin + self.Rs
        Cin = 1 / (2*np.pi*self.fmin*Rin_total)

        # --- C_out Calculation ---
        Zout = self.Rc
        Rout_total = Zout + Rl
        Cout = 1 / (2*np.pi*self.fmin*Rout_total)

        # --- C_E Calculation ---
        # R_parallel_base = Rth || Rs
        if self.Rs == 0:
            R_parallel_base = Rth
        else:
            R_parallel_base = (1/Rth + 1/self.Rs)**-1

        # R_reflected = re_ac + (R_parallel_base / beta)
        R_reflected = re_ac + R_parallel_base / self.B

        # R_CE_total = R_E2 || R_reflected
        R_CE_total = (1/Re2 + 1/R_reflected)**-1
        Ce = 1 / (2 * np.pi * self.fmin * R_CE_total)

        return {'Re1': Re1, 'Re2': Re2, 'Cin': Cin, 'Cout': Cout, 'Ce': Ce}

    # Corrected function signature to include Re2
    def _perform_frequency_analysis(self, Rl, Cin, Cout, Ce, Re1, Re2):
        # Analysis function: Calculates low cutoff frequency given all component values

        # 1. AC Emitter Resistance
        re_ac = 25e-3 / self.Ic

        # 2. Midband Voltage Gain (Am)
        Rc_total = (1/self.Rc + 1/Rl)**-1
        Am = -Rc_total / (re_ac + Re1)
        Am_dB = 20 * np.log10(np.abs(Am))

        # 3. Corner Frequency due to Cin (fL1)
        Rin_transistor = self.B * (re_ac + Re1)
        R_eq_base = (1/self.R1 + 1/self.R2 + 1/Rin_transistor)**-1
        Rin_total = self.Rs + R_eq_base
        fL1 = 1 / (2 * np.pi * Cin * Rin_total)

        # 4. Corner Frequency due to Cout (fL2)
        Rout_collector = self.Rc + Rl
        fL2 = 1 / (2 * np.pi * Cout * Rout_collector)

        # 5. Corner Frequency due to Ce (fLE)
        # R_th_base = Rs || R1 || R2
        R_th_base = (1/self.R1 + 1/self.R2 + 1/self.Rs)**-1

        # R_reflected = re' + R_th_base / beta
        R_reflected = re_ac + R_th_base / self.B

        # Req_emitter = R_E2 || R_reflected
        Req_emitter = (1/Re2 + 1/R_reflected)**-1
        fLE = 1 / (2 * np.pi * Ce * Req_emitter)

        # 6. Total Low Cutoff Frequency (fL)
        fL_total = (fL1**2 + fL2**2 + fLE**2)**0.5

        return {
            'Midband Voltage Gain (Am)': Am,
            'Midband Voltage Gain (dB)': Am_dB,
            'fL1 (Cin)': fL1,
            'fL2 (Cout)': fL2,
            'fLE (Ce)': fLE,
            'Low Cutoff Frequency (fL)': fL_total
            }

In [8]:
calc = CE_AMP_calculator_NF(
    Vcc=12,
    Ic=1e-3,
    Vbe=0.7,
    B=300,
    fmin=20,
    Rs=50
)

In [9]:
resistors = calc._calculate_circuit_resistors()
for k, v in resistors.items():
    print(f'{k}: {round(v/1e3, 2)}kΩ')

Re: 1.2kΩ
Rc: 4.8kΩ
R1: 303.0kΩ
R2: 57.0kΩ


In [10]:
ac_params = calc._calculate_AC_parameters(
    Rl = 1e4,
    Av_new = 20
)
ac_params

{'Re1': np.float64(137.16216216216213),
 'Re2': np.float64(1062.8378378378382),
 'Cin': np.float64(3.28768158964939e-07),
 'Cout': 5.376856185537004e-07,
 'Ce': np.float64(0.00032369131756916755)}

In [11]:
freq_params = calc._perform_frequency_analysis(
    Rl = 1e4,
    Cin = ac_params['Cin'],
    Cout = ac_params['Cout'],
    Ce = ac_params['Ce'],
    Re1 = ac_params['Re1'],
    Re2 = ac_params['Re2']
)
freq_params

{'Midband Voltage Gain (Am)': np.float64(-20.0),
 'Midband Voltage Gain (dB)': np.float64(26.020599913279625),
 'fL1 (Cin)': np.float64(20.0),
 'fL2 (Cout)': 20.0,
 'fLE (Ce)': np.float64(20.0),
 'Low Cutoff Frequency (fL)': np.float64(34.64101615137755)}